<a href="https://colab.research.google.com/github/CER-REC/esa-data-bank_banque-donnees-ees/blob/master/Codes/Section_05_VEC_Labeling_for_Tables/02_labeling_VCs_with_zero_shot_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install tqdm

     |████████████████████████████████| 2.5MB 13.2MB/s 
     |████████████████████████████████| 901kB 21.1MB/s 
     |████████████████████████████████| 3.3MB 44.6MB/s 


In [2]:
import pandas as pd
from transformers import pipeline
from bs4 import BeautifulSoup
import nltk
from nltk.stem.porter import *
from tqdm import tqdm

nltk.download('stopwords', quiet=True)

True

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
df_joined = pd.read_csv('/content/drive/MyDrive/data/esa_index_files/zero_shot_esa_index_train_128_max_tokens.csv', index_col=0)

In [5]:
len(df_joined)

28891

In [6]:
df_joined.head()

,Index,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,CSV Download URL,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,text,label
0,9134,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Co...",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-wor...,1059614_14_lattice-v_1.csv,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABO...,-1
1,9135,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Co...",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_17_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,-1
2,9136,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Co...",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,18,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_18_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,-1
3,9137,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,1059614,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"British Columbia, All",GH-001-2014,"Stantec Consulting Ltd., TERA Environmental Co...",Operating,GC-125,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/1...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,Appendix G: TERA Aquatics Summary Report,15.0,"Water, All",http://www.cer-rec.gc.ca/esa-ees/nrthmntn/nrth...,19,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_19_lattice-v_1.csv,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,-1
4,9138,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,1059614,https://apps.cer-rec

In [7]:
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("typeform/distilbert-base-uncased-mnli")

In [8]:
# os.environ["TOKENIZERS_PARALLELISM"] = "true"

# encoding = tokenizer.encode_plus(
#   sequence_batch,
#   add_special_tokens=True,
#   max_length=MAX_TOKEN_COUNT,
#   return_token_type_ids=False,
#   padding="max_length",
#   truncation=True,
#   return_attention_mask=True,
#   return_tensors='pt',
# )

In [9]:
# Explicitly ask for tensor allocation on CUDA device :0
# classifier = pipeline("zero-shot-classification", model="typeform/distilbert-base-uncased-mnli", tokenizer=, device=0)
classifier = pipeline("zero-shot-classification", device=0)

In [10]:
df_joined.iloc[100]['text']

'Table 425 Major Fire Departments in the Infrastructure and Services in the RAA FulltimeEmployees Equipment ServicesProvided AreasCommunitiesServed HudsonsHope FireDepartment 1 fire chief 25 firefighters Not specified fireprevention andinspectionservices City of Hudsons Hopeand surrounding area MoberlyLake FireDepartment 1 fire chief Severalfirefighters notspecified Not specified Not specified Moberly Lake areaWest Moberly FirstNation and SaulteauFirst Nation SOURCE Modified from PRRD 2013a '

In [11]:
sequences = []

for x in range(len(df_joined)):
  sequences.append(df_joined.iloc[x]['text'][0:250])

In [12]:
len(sequences)

28891

In [13]:
sequences[0]

'TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABORIGINAL FIELD STUDY PARTICIPATION FOR THE PROJECT Survey Date Aboriginal Communities Detail July 9 to 12 2011 Blueberry River First Nation Halfway River First Nation McLeod Lake Indian Band North East Mti'

In [14]:
candidate_labels = """Physical and meteorological environment,Soil and soil productivity,Vegetation,Water quality and quantity,Fish and fish habitat,Wetlands,Wildlife and wildlife habitat,Species at Risk or Species of Special Status and related habitat,Greenhouse gas (GHG) emissions and climate change,GHG Emissions and Climate Change – Assessment of Upstream GHG Emissions,Air emissions,Acoustic environment,Electromagnetism and Corona Discharge,Human occupancy and resource use,Heritage resources,Navigation and navigation safety,Traditional land and resource use,Social and cultural well-being,Human health and aesthetics,Infrastructure and services,Employment and economy,Environmental Obligations,Rights of Indigenous Peoples""".split(',')
hypothesis_template = "The data from this table is about {}."
candidate_labels

['Physical and meteorological environment',
 'Soil and soil productivity',
 'Vegetation',
 'Water quality and quantity',
 'Fish and fish habitat',
 'Wetlands',
 'Wildlife and wildlife habitat',
 'Species at Risk or Species of Special Status and related habitat',
 'Greenhouse gas (GHG) emissions and climate change',
 'GHG Emissions and Climate Change – Assessment of Upstream GHG Emissions',
 'Air emissions',
 'Acoustic environment',
 'Electromagnetism and Corona Discharge',
 'Human occupancy and resource use',
 'Heritage resources',
 'Navigation and navigation safety',
 'Traditional land and resource use',
 'Social and cultural well-being',
 'Human health and aesthetics',
 'Infrastructure and services',
 'Employment and economy',
 'Environmental Obligations',
 'Rights of Indigenous Peoples']

In [15]:
import gc
import torch

gc.collect()

torch.cuda.empty_cache()

In [16]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |    1553 MB |    1553 MB |    1553 MB |       0 B  |\n|       from large pool |    1552 MB |    1552 MB |    1552 MB |       0 B  |\n|       from small pool |       1 MB |       1 MB |       1 MB |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |    1553 MB |    1553 MB |    1553 MB |       0 B  |\n|       from large pool |    1552 MB |    1552 MB |

In [17]:
from itertools import zip_longest
 
def grouper(iterable_obj, count, fillvalue=None):
    args = [iter(iterable_obj)] * count
    return zip_longest(*args, fillvalue=fillvalue)

output = [] 

for sequence_batch in tqdm(grouper(sequences, 2, "")):
    # print(sequence_batch)
    output.append(classifier(sequence_batch, candidate_labels, hypothesis_template=hypothesis_template, multi_label=True))

14446it [1:54:10,  2.11it/s]


In [18]:
output[0]

[{'labels': ['Water quality and quantity',
   'Traditional land and resource use',
   'Environmental Obligations',
   'Rights of Indigenous Peoples',
   'Heritage resources',
   'Social and cultural well-being',
   'Infrastructure and services',
   'Species at Risk or Species of Special Status and related habitat',
   'Human occupancy and resource use',
   'GHG Emissions and Climate Change – Assessment of Upstream GHG Emissions',
   'Soil and soil productivity',
   'Physical and meteorological environment',
   'Vegetation',
   'Fish and fish habitat',
   'Wetlands',
   'Employment and economy',
   'Human health and aesthetics',
   'Wildlife and wildlife habitat',
   'Greenhouse gas (GHG) emissions and climate change',
   'Acoustic environment',
   'Air emissions',
   'Navigation and navigation safety',
   'Electromagnetism and Corona Discharge'],
  'scores': [0.7399547696113586,
   0.05952973663806915,
   0.017496826127171516,
   0.012848653830587864,
   0.005223226733505726,
   0.0031

In [19]:
# Since we have a list of lists for the outputs (because inference was done in batches), we need to flatten the lists together into one.
flatOutput = [item for elem in output for item in elem]

In [20]:
pickled_dataset = 'zero_shot_vcs_train'

In [21]:
import pickle 
with open(f'{pickled_dataset}.pkl', 'wb') as f:
  pickle.dump(flatOutput, f)

In [22]:
import pickle
infile = open(f'{pickled_dataset}.pkl', 'rb')
zs_vcs_dict_list = pickle.load(infile)
infile.close()

In [23]:
zs_vcs_dict_list[0]

{'labels': ['Water quality and quantity',
  'Traditional land and resource use',
  'Environmental Obligations',
  'Rights of Indigenous Peoples',
  'Heritage resources',
  'Social and cultural well-being',
  'Infrastructure and services',
  'Species at Risk or Species of Special Status and related habitat',
  'Human occupancy and resource use',
  'GHG Emissions and Climate Change – Assessment of Upstream GHG Emissions',
  'Soil and soil productivity',
  'Physical and meteorological environment',
  'Vegetation',
  'Fish and fish habitat',
  'Wetlands',
  'Employment and economy',
  'Human health and aesthetics',
  'Wildlife and wildlife habitat',
  'Greenhouse gas (GHG) emissions and climate change',
  'Acoustic environment',
  'Air emissions',
  'Navigation and navigation safety',
  'Electromagnetism and Corona Discharge'],
 'scores': [0.7399547696113586,
  0.05952973663806915,
  0.017496826127171516,
  0.012848653830587864,
  0.005223226733505726,
  0.003144525922834873,
  0.002707538

In [24]:
len(zs_vcs_dict_list)

28892

In [ ]:
for i in range(0, len(zs_vcs_dict_list)):
  zs_vcs_dict_list[0]['scores']

{'labels': ['Traditional land and resource use',
  'Water quality and quantity',
  'Social and cultural well-being',
  'Human occupancy and resource use',
  'Fish and fish habitat',
  'Heritage resources',
  'Navigation and navigation safety',
  'Vegetation',
  'Wetlands',
  'Physical and meteorological environment',
  'Soil and soil productivity',
  'Infrastructure',
  'Wildlife and wildlife habitat',
  'Species at Risk or Species of Special Status and related habitat',
  'Human health and aesthetics',
  'Electromagnetism and Corona Discharge',
  'Acoustic environment',
  'Air emissions',
  'Greenhouse gas (GHG) emissions and climate change'],
 'scores': [0.6877259612083435,
  0.649422287940979,
  0.4630372226238251,
  0.30576542019844055,
  0.08918734639883041,
  0.03485028073191643,
  0.03348839655518532,
  0.02901163510978222,
  0.017860960215330124,
  0.015356151387095451,
  0.012270933017134666,
  0.009136791341006756,
  0.008778736926615238,
  0.006375862751156092,
  0.004991511

In [ ]:
df_output = pd.DataFrame(output)
df_output

{'labels': ['Traditional land and resource use',
  'Water quality and quantity',
  'Social and cultural well-being',
  'Human occupancy and resource use',
  'Fish and fish habitat',
  'Heritage resources',
  'Navigation and navigation safety',
  'Vegetation',
  'Wetlands',
  'Physical and meteorological environment',
  'Soil and soil productivity',
  'Infrastructure',
  'Wildlife and wildlife habitat',
  'Species at Risk or Species of Special Status and related habitat',
  'Human health and aesthetics',
  'Electromagnetism and Corona Discharge',
  'Acoustic environment',
  'Air emissions',
  'Greenhouse gas (GHG) emissions and climate change'],
 'scores': [0.6877259612083435,
  0.649422287940979,
  0.4630372226238251,
  0.30576542019844055,
  0.08918734639883041,
  0.03485028073191643,
  0.03348839655518532,
  0.02901163510978222,
  0.017860960215330124,
  0.015356151387095451,
  0.012270933017134666,
  0.009136791341006756,
  0.008778736926615238,
  0.006375862751156092,
  0.004991511

In [ ]:
df_output[df_output.scores > 0.6]